# Preparação do ambiente

In [ ]:
import pandas as pd
import zipfile
import matplotlib.pyplot as plt
import numpy as np
import glob
import keras
import seaborn as sns
import scipy as sp
from scipy import signal
from scipy.signal import hilbert

In [ ]:
edf_format = True
if edf_format:
  !wget -P "./EEG-data" https://physionet.org/static/published-projects/eegmat/eeg-during-mental-arithmetic-tasks-1.0.0.zip
  !pip install pyEDFlib
  import pyedflib
else:
  !gdown '1pRE91UrrutDNPdSCzjsf64DgiKl25k8d'
  !unzip "/content/CSVs.zip" -d "/content/CSVs"
  !rm -rf '/content/CSVs.zip'

--2022-10-03 16:18:40--  https://physionet.org/static/published-projects/eegmat/eeg-during-mental-arithmetic-tasks-1.0.0.zip
Resolving physionet.org (physionet.org)... 18.18.42.54
Connecting to physionet.org (physionet.org)|18.18.42.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 183634285 (175M) [application/zip]
Saving to: ‘./EEG-data/eeg-during-mental-arithmetic-tasks-1.0.0.zip’

eeg-during-mental-a 100%[===================>] 175.13M  3.72MB/s    in 44s     

2022-10-03 16:19:24 (3.95 MB/s) - ‘./EEG-data/eeg-during-mental-arithmetic-tasks-1.0.0.zip’ saved [183634285/183634285]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Funções utilizadas

In [ ]:
def segmenta(sinal, passo, dimensoes):
  #Inputs
  #- sinal: sinal a ser segmentado
  #- passo: quantas amostras irá avançar de um segmento para outro
  #- dimensoes: Dimensões esperada do output

  #Output
  #- segmentado: Sinal segmentado em segmentos de 2*passo de duração e com
  # overlap de 1 passo entre eles

  segmentado = np.zeros(dimensoes)
  cont = 0
  for linha in range(sinal.shape[0]):
      sinal_s = sinal[linha,:,:]
      for i in range(1000000):
          if ~np.any(sinal_s[i*passo:(i+2)*passo,:]) or sinal_s[i*passo:(i+2)*passo,:].shape[0] != int(2*passo):
              break
          segmentado[cont,:,:] = sinal_s[i*passo:(i+2)*passo,:]
          cont = cont+1
  return segmentado

def Features(sinal,fs=500):
  #Inputs
  #- sinal: sinal de onde serão tiradas as características
  #- fs: frequência de amostragem do sinal

  #Output
  #- sig_features: 

  sig_features = np.zeros((256,20))

  # A extração de características é feita canal por canal
  for i in range(20):
    sinal_c = sinal[:,i]

    # Encontra a frequência instantanea do canal utilizando a transformada de Hilbert
    analytic_signal = hilbert(sinal_c)
    instantaneous_phase = np.unwrap(np.angle(analytic_signal))
    instantaneous_frequency = np.abs((np.diff(instantaneous_phase) / (2.0*np.pi) * fs))
    instantaneous_frequency = instantaneous_frequency.reshape((-1,1))
    

    # Determina a densidade do espectro de potência do canal
    f, psd = signal.welch(sinal_c, 500)

    alfa = np.trapz(y = psd[4:6], x = f[4:6]).reshape((-1,1))
    beta = np.trapz(y = psd[6:15], x = f[6:15]).reshape((-1,1))
    gamma = np.trapz(y = psd[15:50], x = f[15:50]).reshape((-1,1))
    delta = np.trapz(y = psd[0:2], x = f[0:2]).reshape((-1,1))
    theta = np.trapz(y = psd[2:4], x = f[2:4]).reshape((-1,1))

    sig_mean = np.mean(sinal_c).reshape((-1,1))

    sig_std = np.std(sinal_c).reshape((-1,1))

    feat = np.concatenate((instantaneous_frequency,delta,theta,alfa,beta,gamma,sig_mean,sig_std)).reshape((-1,1))

    sig_features[:,i] = feat[:,0]
  
  return sig_features

# Preparação dos dados

## Caso esteja pegando sinal dos arquivos .edf

In [ ]:
if edf_format:
  with zipfile.ZipFile("/EEG-data/eeg-during-mental-arithmetic-tasks-1.0.0.zip") as z:
    lista_arquivos = z.namelist()
  
  lista_arquivos.remove('eeg-during-mental-arithmetic-tasks-1.0.0/subject-info.csv')
  lista_arquivos.remove('eeg-during-mental-arithmetic-tasks-1.0.0/SHA256SUMS.txt')
  lista_arquivos.remove('eeg-during-mental-arithmetic-tasks-1.0.0/README.txt')
  lista_arquivos.remove('eeg-during-mental-arithmetic-tasks-1.0.0/RECORDS')

In [ ]:
if edf_format:
  repouso = []
  ativos = []

  for i in lista_arquivos:
      if(i.endswith('_2.edf')):
          ativos.append(i)
      else:
          repouso.append(i)

  with zipfile.ZipFile("./EEG-data/eeg-during-mental-arithmetic-tasks-1.0.0.zip") as z:
      arq = pyedflib.EdfReader(z.extract(repouso[0]))
      labels = arq.getSignalLabels()
      arq.close()
  labels.pop()

  ##Coletando os dados de antes as perguntas
  dfs_repouso = [None] * 36
  with zipfile.ZipFile("./EEG-data/eeg-during-mental-arithmetic-tasks-1.0.0.zip") as z:
      for i in repouso:
          arq = pyedflib.EdfReader(z.extract(i))
          j = int(i[-8]+i[-7])
          x = pd.DataFrame()
          for k in range(0,len(labels)):
              x[labels[k]] = arq.readSignal(k)
          dfs_repouso[j] = x
          arq.close()

  ##Coletando os dados de durante as perguntas
  dfs_ativos = [None] * 36
  with zipfile.ZipFile("./EEG-data/eeg-during-mental-arithmetic-tasks-1.0.0.zip") as z:
      for i in ativos:
          arq = pyedflib.EdfReader(z.extract(i))
          j = int(i[-8]+i[-7])
          x = pd.DataFrame()
          for k in range(0,len(labels)):
              x[labels[k]] = arq.readSignal(k)
          dfs_ativos[j] = x    
          arq.close()

In [ ]:
if edf_format:
  mult = int(5e3) #Nº de amostras na intersecção
  seq_rest = 0
  seq_active = 0

  # Nome de cada coluna do dataframe (para referência)
  # nomes = ['Fp1','Fp2','F3','F4','F7','F8','T3','T4','C3','C4','T5','T6','P3','P4','O1','O2','Fz','Cz','Pz','A2-A1','ECG','State']
  nomes = ['Fp1','Fp2','F3','F4','F7','F8','T3','T4','C3','C4','T5','T6','P3','P4','O1','O2','Fz','Cz','Pz','A2-A1']

  # A quantidade de cada tipo de segmento definida com base no artigo
  rest_seqs = np.zeros((601,10000,len(nomes)))
  acti_seqs = np.ones((180,10000,len(nomes)))

  for ativo in dfs_ativos:
    aux = ativo.to_numpy()
    for i in range(1000000):
      # Quando a segmentação resultar em um segmento com menos de 10k de amostras ou resulte em
      # um vetor vazio (o indice de início é maior que o vetor), acaba a segmentação deste arquivo
      # e vai para o próximo.
      if ~np.any(aux[i*mult:(i+2)*mult,:]) or aux[i*mult:(i+2)*mult,:].shape[0] != 10000:
        break
      acti_seqs[seq_active,:,:] = aux[i*mult:(i+2)*mult,:]
      seq_active = seq_active+1

  for repouso in dfs_repouso:
    aux = repouso.to_numpy()
    for i in range(1000000):
      # Quando a segmentação resultar em um segmento com menos de 10k de amostras ou resulte em
      # um vetor vazio (o indice de início é maior que o vetor), acaba a segmentação deste arquivo
      # e vai para o próximo.
      if ~np.any(aux[i*mult:(i+2)*mult,:]) or aux[i*mult:(i+2)*mult,:].shape[0] != 10000:
        break
      rest_seqs[seq_rest,:,:] = aux[i*mult:(i+2)*mult,:]
      seq_rest = seq_rest+1

  rest_seqs = np.concatenate((rest_seqs,np.zeros((601,10000,1))),axis=2)
  acti_seqs = np.concatenate((acti_seqs,np.ones((180,10000,1))),axis=2)

## Caso esteja pegando o sinal dos arquivos .csv

In [ ]:
if not edf_format:
  files = glob.glob("./CSVs/*.csv")

  row = 0
  mult = int(5e3) #Nº de amostras na intersecção
  seq_rest = 0
  seq_active = 0

  # Nome de cada coluna do dataframe (para referência)
  nomes = ['Fp1','Fp2','F3','F4','F7','F8','T3','T4','C3','C4','T5','T6','P3','P4','O1','O2','Fz','Cz','Pz','A2-A1','ECG','State']

  # A quantidade de cada tipo de segmento definida com base no artigo
  rest_seqs = np.zeros((601,10000,len(nomes)))
  acti_seqs = np.ones((180,10000,len(nomes)))

  for file in files:
    df = pd.read_csv(file)
    subject = int(file[-5])-1
    df['State'] = subject

    aux = df.to_numpy()
    
    for i in range(1000000):
      # Quando a segmentação resultar em um segmento com menos de 10k de amostras ou resulte em
      # um vetor vazio (o indice de início é maior que o vetor), acaba a segmentação deste arquivo
      # e vai para o próximo.
      if ~np.any(aux[i*mult:(i+2)*mult,:]) or aux[i*mult:(i+2)*mult,:].shape[0] != 10000:
        break
      if subject==1:
        acti_seqs[seq_active,:,:] = aux[i*mult:(i+2)*mult,:]
        seq_active = seq_active+1
      else:
        rest_seqs[seq_rest,:,:] = aux[i*mult:(i+2)*mult,:]
        seq_rest = seq_rest+1

## Separação em conjunto de treino e de teste

In [ ]:
rest_train = list(np.random.choice(601,480,replace=False))
rest_test = [x for x in range(601) if x not in rest_train]

active_train = list(np.random.choice(180,144,replace=False))
active_test = [x for x in range(180) if x not in active_train]

rest_train = rest_seqs[rest_train,:,:]
active_train = acti_seqs[active_train,:,:]
rest_test = rest_seqs[rest_test,:,:]
active_test = acti_seqs[active_test,:,:]

## Segmentação do sinal em trechos de 0.5s

In [ ]:
rest_train = segmenta(rest_train,125,(37920,250,22))
active_train = segmenta(active_train,125,(11376,250,22))
rest_test = segmenta(rest_test,125,(9559,250,22))
active_test = segmenta(active_test,125,(2844,250,22))

In [ ]:
train = np.concatenate((rest_train,active_train),axis=0)
test = np.concatenate((rest_test,active_test),axis=0)

In [ ]:
train_data, y_train = train[:,:,:-1], train[:,:,-1]
test_data, y_test = test[:,:,:-1], test[:,:,-1]

In [ ]:
y_train = np.mean(y_train, axis=1,dtype=np.int16).reshape((-1,1))
y_test = np.mean(y_test, axis=1,dtype=np.int16).reshape((-1,1))

# Extração das características

In [ ]:
x_train = np.zeros((train_data.shape[0],256,20))
for segm in range(train_data.shape[0]):
    x_train[segm,:,:] = Features(train_data[segm,:,:])

x_test = np.zeros((test_data.shape[0],256,20))
for segm in range(test_data.shape[0]):
    x_test[segm,:,:] = Features(test_data[segm,:,:])

In [ ]:
np.savez_compressed('Train',data = x_train, labels = y_train)
np.savez_compressed('Test',data = x_test, labels = y_test)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp '/content/Train.npz' '/content/drive/MyDrive'
!cp '/content/Test.npz' '/content/drive/MyDrive'